In [2]:
import matplotlib.pyplot as plt
import os
import pandas as pd

In [24]:
output_dir = "graficos_adosamiento_edificio"
os.makedirs(output_dir, exist_ok=True)

In [3]:
nepal = pd.read_csv("eq2015.csv")
fuente_1 = "https://www.kaggle.com/datasets/sanskarnegi/nepal-earthquake-2015"

In [26]:
distritos = nepal["district_name"].unique()
distritos

array(['OKHALDHUNGA', 'SINDHULI', 'RAMECHHAP', 'DOLAKHA', 'SINDHUPALCHOK',
       'KAVRE', 'NUWAKOT', 'RASUWA', 'DHADING', 'MAKWANPUR', 'GORKHA'],
      dtype=object)

In [27]:
graficos_por_distrito = {}

In [60]:
for distrito in distritos:
    df_distrito = nepal[nepal["district_name"] == distrito]
    df_grouped = df_distrito.groupby(["position", "damage_grade"]).size().unstack(fill_value=0)
    
    # Asegurar presencia de todos los grados de daño
    todos_los_grados = ["Grade 1", "Grade 2", "Grade 3", "Grade 4", "Grade 5"]
    df_grouped = df_grouped.reindex(columns=todos_los_grados, fill_value=0)
    
    orden_adosamiento = ["Not attached", "Attached-1 side", "Attached-2 side", "Attached-3 side"]
    df_grouped = df_grouped.reindex(orden_adosamiento,fill_value=0)

    # Normalizar las filas para que cada barra tenga altura total 1
    df_normalized = df_grouped.div(df_grouped.sum(axis=1), axis=0).fillna(0)

    # Crear la figura
    fig, ax = plt.subplots(figsize=(8, 10))
    df_normalized.plot(kind='bar', stacked=True, colormap='viridis', ax=ax,legend=False)

    # Etiquetas
    plt.xlabel('Tipo de adosamiento')
    plt.ylabel('Número de edificios')
    plt.title(f'Distrito: {distrito}')
    plt.xticks(rotation=0)
    ax.set_ylim(0, 1)

    for i, (idx, row) in enumerate(df_normalized.iterrows()):
        y_offset = 0
        for damage_grade, value in row.items():
            if value > 0:
                ax.text(
                    i + 0.48,
                    y_offset + value / 2, 
                    f"{value * 100:.1f}%", 
                    ha='right', 
                    va='center',
                    fontsize=7,
                    color='black'
                )
            y_offset += value
    # Guardar la imagen en memoria
    img_path = os.path.join(output_dir, f"{distrito}.png")
    plt.savefig(img_path, bbox_inches="tight", dpi=150)
    plt.close(fig)

    # Guardar ruta de la imagen
    graficos_por_distrito[distrito] = img_path

In [6]:
lista_gorkha = nepal[nepal["district_name"] == "GORKHA"]

In [9]:
from scipy.stats import chi2_contingency, fisher_exact

# Función para hacer el análisis entre dos tipos de adosamiento
def analizar_contraste(position_a, position_b):
    print(f"\n📊 Comparando: {position_a} vs {position_b}")
    
    # Filtrar los dos grupos
    subset = lista_gorkha[lista_gorkha["position"].isin([position_a, position_b])].copy()

    # ------------------------------
    # 1. Test de Chi-cuadrado completo
    # ------------------------------
    tabla_chi = pd.crosstab(subset["position"], subset["damage_grade"])
    chi2, p_chi2, dof, expected = chi2_contingency(tabla_chi)
    
    print(f"\n🔍 Test de Chi-cuadrado:")
    print(f"Chi² = {chi2:.2f}, dof = {dof}, p-valor = {p_chi2:.4e}")
    
    # ------------------------------
    # 2. Test exacto de Fisher (colapsado vs no colapsado)
    # ------------------------------
    subset["colapsado"] = subset["damage_grade"] == "Grade 5"
    tabla_fisher = pd.crosstab(subset["position"], subset["colapsado"])
    
    # Solo se puede aplicar si la tabla es 2x2
    if tabla_fisher.shape == (2, 2):
        oddsratio, p_fisher = fisher_exact(tabla_fisher)
        print(f"\n🔬 Test exacto de Fisher (Grade 5 vs resto):")
        print(f"Odds Ratio = {oddsratio:.2f}, p-valor = {p_fisher:.4e}")
    else:
        print("⚠️ No se puede aplicar test de Fisher (tabla no 2x2).")

# Comparaciones deseadas
analizar_contraste("Not attached", "Attached-1 side")
analizar_contraste("Not attached", "Attached-2 side")
analizar_contraste("Not attached", "Attached-3 side")


📊 Comparando: Not attached vs Attached-1 side

🔍 Test de Chi-cuadrado:
Chi² = 30.70, dof = 4, p-valor = 3.5211e-06

🔬 Test exacto de Fisher (Grade 5 vs resto):
Odds Ratio = 1.00, p-valor = 8.9195e-01

📊 Comparando: Not attached vs Attached-2 side

🔍 Test de Chi-cuadrado:
Chi² = 845.80, dof = 4, p-valor = 9.2148e-182

🔬 Test exacto de Fisher (Grade 5 vs resto):
Odds Ratio = 1.46, p-valor = 1.1535e-21

📊 Comparando: Not attached vs Attached-3 side

🔍 Test de Chi-cuadrado:
Chi² = 34.73, dof = 4, p-valor = 5.2887e-07

🔬 Test exacto de Fisher (Grade 5 vs resto):
Odds Ratio = 3.08, p-valor = 3.9732e-05


In [10]:
lista_dolakha = nepal[nepal["district_name"] == "DOLAKHA"]

In [11]:
# Función para hacer el análisis entre dos tipos de adosamiento
def analizar_contraste(position_a, position_b):
    print(f"\n📊 Comparando: {position_a} vs {position_b}")
    
    # Filtrar los dos grupos
    subset = lista_dolakha[lista_dolakha["position"].isin([position_a, position_b])].copy()

    # ------------------------------
    # 1. Test de Chi-cuadrado completo
    # ------------------------------
    tabla_chi = pd.crosstab(subset["position"], subset["damage_grade"])
    chi2, p_chi2, dof, expected = chi2_contingency(tabla_chi)
    
    print(f"\n🔍 Test de Chi-cuadrado:")
    print(f"Chi² = {chi2:.2f}, dof = {dof}, p-valor = {p_chi2:.4e}")
    
    # ------------------------------
    # 2. Test exacto de Fisher (colapsado vs no colapsado)
    # ------------------------------
    subset["colapsado"] = subset["damage_grade"] == "Grade 5"
    tabla_fisher = pd.crosstab(subset["position"], subset["colapsado"])
    
    # Solo se puede aplicar si la tabla es 2x2
    if tabla_fisher.shape == (2, 2):
        oddsratio, p_fisher = fisher_exact(tabla_fisher)
        print(f"\n🔬 Test exacto de Fisher (Grade 5 vs resto):")
        print(f"Odds Ratio = {oddsratio:.2f}, p-valor = {p_fisher:.4e}")
    else:
        print("⚠️ No se puede aplicar test de Fisher (tabla no 2x2).")

# Comparaciones deseadas
analizar_contraste("Not attached", "Attached-1 side")
analizar_contraste("Not attached", "Attached-2 side")
analizar_contraste("Not attached", "Attached-3 side")


📊 Comparando: Not attached vs Attached-1 side

🔍 Test de Chi-cuadrado:
Chi² = 46.94, dof = 4, p-valor = 1.5713e-09

🔬 Test exacto de Fisher (Grade 5 vs resto):
Odds Ratio = 1.14, p-valor = 9.0507e-08

📊 Comparando: Not attached vs Attached-2 side

🔍 Test de Chi-cuadrado:
Chi² = 656.28, dof = 4, p-valor = 1.0175e-140

🔬 Test exacto de Fisher (Grade 5 vs resto):
Odds Ratio = 1.89, p-valor = 3.4218e-24

📊 Comparando: Not attached vs Attached-3 side

🔍 Test de Chi-cuadrado:
Chi² = 10.09, dof = 4, p-valor = 3.8965e-02

🔬 Test exacto de Fisher (Grade 5 vs resto):
Odds Ratio = 2.88, p-valor = 8.2637e-02


In [12]:
lista_makwanpur = nepal[nepal["district_name"] == "MAKWANPUR"]

In [13]:
# Función para hacer el análisis entre dos tipos de adosamiento
def analizar_contraste(position_a, position_b):
    print(f"\n📊 Comparando: {position_a} vs {position_b}")
    
    # Filtrar los dos grupos
    subset = lista_makwanpur[lista_makwanpur["position"].isin([position_a, position_b])].copy()

    # ------------------------------
    # 1. Test de Chi-cuadrado completo
    # ------------------------------
    tabla_chi = pd.crosstab(subset["position"], subset["damage_grade"])
    chi2, p_chi2, dof, expected = chi2_contingency(tabla_chi)
    
    print(f"\n🔍 Test de Chi-cuadrado:")
    print(f"Chi² = {chi2:.2f}, dof = {dof}, p-valor = {p_chi2:.4e}")
    
    # ------------------------------
    # 2. Test exacto de Fisher (colapsado vs no colapsado)
    # ------------------------------
    subset["colapsado"] = subset["damage_grade"] == "Grade 5"
    tabla_fisher = pd.crosstab(subset["position"], subset["colapsado"])
    
    # Solo se puede aplicar si la tabla es 2x2
    if tabla_fisher.shape == (2, 2):
        oddsratio, p_fisher = fisher_exact(tabla_fisher)
        print(f"\n🔬 Test exacto de Fisher (Grade 5 vs resto):")
        print(f"Odds Ratio = {oddsratio:.2f}, p-valor = {p_fisher:.4e}")
    else:
        print("⚠️ No se puede aplicar test de Fisher (tabla no 2x2).")

# Comparaciones deseadas
analizar_contraste("Not attached", "Attached-1 side")
analizar_contraste("Not attached", "Attached-2 side")
analizar_contraste("Not attached", "Attached-3 side")


📊 Comparando: Not attached vs Attached-1 side

🔍 Test de Chi-cuadrado:
Chi² = 194.39, dof = 4, p-valor = 6.0271e-41

🔬 Test exacto de Fisher (Grade 5 vs resto):
Odds Ratio = 1.42, p-valor = 1.1190e-19

📊 Comparando: Not attached vs Attached-2 side

🔍 Test de Chi-cuadrado:
Chi² = 219.30, dof = 4, p-valor = 2.6472e-46

🔬 Test exacto de Fisher (Grade 5 vs resto):
Odds Ratio = 3.85, p-valor = 6.2224e-52

📊 Comparando: Not attached vs Attached-3 side

🔍 Test de Chi-cuadrado:
Chi² = 30.39, dof = 4, p-valor = 4.0708e-06

🔬 Test exacto de Fisher (Grade 5 vs resto):
Odds Ratio = 2.79, p-valor = 6.9901e-04


In [14]:
lista_rasuwa = nepal[nepal["district_name"] == "RASUWA"]

In [15]:
# Función para hacer el análisis entre dos tipos de adosamiento
def analizar_contraste(position_a, position_b):
    print(f"\n📊 Comparando: {position_a} vs {position_b}")
    
    # Filtrar los dos grupos
    subset = lista_rasuwa[lista_rasuwa["position"].isin([position_a, position_b])].copy()

    # ------------------------------
    # 1. Test de Chi-cuadrado completo
    # ------------------------------
    tabla_chi = pd.crosstab(subset["position"], subset["damage_grade"])
    chi2, p_chi2, dof, expected = chi2_contingency(tabla_chi)
    
    print(f"\n🔍 Test de Chi-cuadrado:")
    print(f"Chi² = {chi2:.2f}, dof = {dof}, p-valor = {p_chi2:.4e}")
    
    # ------------------------------
    # 2. Test exacto de Fisher (colapsado vs no colapsado)
    # ------------------------------
    subset["colapsado"] = subset["damage_grade"] == "Grade 5"
    tabla_fisher = pd.crosstab(subset["position"], subset["colapsado"])
    
    # Solo se puede aplicar si la tabla es 2x2
    if tabla_fisher.shape == (2, 2):
        oddsratio, p_fisher = fisher_exact(tabla_fisher)
        print(f"\n🔬 Test exacto de Fisher (Grade 5 vs resto):")
        print(f"Odds Ratio = {oddsratio:.2f}, p-valor = {p_fisher:.4e}")
    else:
        print("⚠️ No se puede aplicar test de Fisher (tabla no 2x2).")

# Comparaciones deseadas
analizar_contraste("Not attached", "Attached-1 side")
analizar_contraste("Not attached", "Attached-2 side")
analizar_contraste("Not attached", "Attached-3 side")


📊 Comparando: Not attached vs Attached-1 side

🔍 Test de Chi-cuadrado:
Chi² = 81.07, dof = 4, p-valor = 1.0332e-16

🔬 Test exacto de Fisher (Grade 5 vs resto):
Odds Ratio = 1.42, p-valor = 2.4465e-15

📊 Comparando: Not attached vs Attached-2 side

🔍 Test de Chi-cuadrado:
Chi² = 161.71, dof = 4, p-valor = 6.2808e-34

🔬 Test exacto de Fisher (Grade 5 vs resto):
Odds Ratio = 2.10, p-valor = 8.6751e-34

📊 Comparando: Not attached vs Attached-3 side

🔍 Test de Chi-cuadrado:
Chi² = 13.74, dof = 4, p-valor = 8.1852e-03

🔬 Test exacto de Fisher (Grade 5 vs resto):
Odds Ratio = 5.50, p-valor = 2.0146e-02
